In [1]:
from isaacgym import gymtorch
from isaacgym import gymapi
from utils.rlgames_utils import get_rlgames_env_creator
import train
from rl_games.torch_runner import Runner

from omegaconf import OmegaConf
from hydra import compose, initialize
import yaml
from utils import reformat
from isaacgymenvs.utils.rlgames_utils import RLGPUAlgoObserver
import torch
import numpy as np
import time

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.8.1
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...


2022-06-29 15:40:51,761 - INFO - logger - logger initialized


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


In [3]:
import os

os.environ['DISPLAY'] = ":0"

In [4]:
with initialize(config_path="cfg", job_name="test_env"):
    cfg = compose(config_name="config", overrides=["task.env.numEnvs=2", "task.env.observationType=full", "test=true"])

env_loader = get_rlgames_env_creator(
    reformat.omegaconf_to_dict(cfg.task), 'AllegroHand', cfg.sim_device,
    cfg.rl_device, cfg.graphics_device_id, cfg.headless)

env = env_loader()

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:413: UserWarning: In config: Invalid overriding of hydra/job_logging:
Default list overrides requires 'override' keyword.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/defaults_list_override for more information.

  deprecation_warning(msg)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


Obs type: full
[Warning] [carb.gym.plugin] useGpu is set, forcing single scene (0 subscenes)
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
Num dofs:  16
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
Num dofs:  20


In [5]:
def get_action():
    return torch.tensor(np.stack([env.action_space.sample() for _ in range(env.num_envs)]), device=env.device)

In [6]:
obs = env.reset()
action = get_action()
obs, r, done, info = env.step(action)

## Get camera transform
```python
transform = env.gym.get_viewer_camera_transform(env.viewer, env.envs[0])
pos = transform.p
print("position", pos.x, pos.y, pos.z)

rot = transform.r
print("rotation", rot.x, rot.y, rot.z, rot.w)

import pickle
pickle.dump(transform, open('allegro_viewer.pkl', 'wb'))
```

In [ ]:
env.gym.get_viewer_camera_handle(env.viewer)

In [15]:
import pickle

transform = pickle.load(open('allegro_viewer.pkl', 'rb'))
env.gym.set_camera_transform(env.viewer, env.envs[0], transform)

TypeError: set_camera_transform(): incompatible function arguments. The following argument types are supported:
    1. (self: isaacgym._bindings.linux-x86_64.gym_37.Gym, arg0: int, arg1: isaacgym._bindings.linux-x86_64.gym_37.Env, arg2: isaacgym._bindings.linux-x86_64.gym_37.Transform) -> None

Invoked with: <isaacgym._bindings.linux-x86_64.gym_37.Gym object at 0x7f71a317a170>, <isaacgym._bindings.linux-x86_64.gym_37.Viewer object at 0x7f71a32a5ef0>, <isaacgym._bindings.linux-x86_64.gym_37.Env object at 0x7f71a330b330>, <isaacgym._bindings.linux-x86_64.gym_37.Transform object at 0x7f7075cc20f0>

In [7]:
while True:
    action = get_action()
    obs, r, done, info = env.step(action)
    env_ids = done.nonzero(as_tuple=False).squeeze(-1)
    goal_env_ids = env.reset_goal_buf.nonzero(as_tuple=False).squeeze(-1)
    time.sleep(0.01)
    if done.any():
        env.reset_idx(env_ids, goal_env_ids)

KeyboardInterrupt: 